In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("../Dataset/trainable.csv", index_col=0)
df.sample(12)

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.404496,1.0
806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-3.533917,0.0
275,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.432103,1.0
539,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.012263,1.0
355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.610700,0.0
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.885193,1.0
809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.078099,1.0
16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.504553,0.0
309,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.143063,1.0
209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.565169,1.0


In [5]:
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [6]:
import optuna 

c:\Users\skmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
df.iloc[707,43]

np.float64(1.0)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df.drop(columns="43", axis=1), df.iloc[:,43], random_state=0, test_size=0.2)

In [34]:
## Got optimised code from gemini, sadly no better results

# def lr_objective(trial):

#     c = trial.suggest_float('C', 1.0, 100.0)
#     solver = trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'saga'])
#     penalty = trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet'])

#     if penalty == 'elasticnet':                                     #This is to ensure the compatibility between penalties and solversh
#         solver = 'saga'
#         l1_ratio = trial.suggest_float('l1_ratio', 0, 1)
#     elif penalty == 'l1':
#         solver = trial.suggest_categorical('solver_l1', ['liblinear', 'saga'])
#         l1_ratio = None 
#     else: 
#         solver = trial.suggest_categorical('solver_l2', ['lbfgs', 'liblinear', 'sag', 'saga'])
#         l1_ratio = None

#     model = LogisticRegression(

#         penalty = penalty,
#         C = c,
#         l1_ratio = l1_ratio,
#         solver = solver,
#         max_iter = 10000,
#         n_jobs = -1

#     )

#     score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

#     return score

In [13]:
def lr_objective(trial):

    c = trial.suggest_float('C', 1.0, 100.0)
    l1_ratio = trial.suggest_float('l1_ratio', 0, 1)

    model = LogisticRegression(

        penalty='elasticnet',
        C = c,
        l1_ratio= l1_ratio,
        solver = 'saga',
        max_iter=10000,
        n_jobs=-1

    )

    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score

In [14]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(lr_objective, n_trials=50)

[I 2026-02-22 19:13:47,188] A new study created in memory with name: no-name-d5867fd9-f94c-46e6-889e-323ae7c58bc6
[I 2026-02-22 19:13:47,962] Trial 0 finished with value: 0.8207834400816857 and parameters: {'C': 85.14571956554886, 'l1_ratio': 0.13682410559218816}. Best is trial 0 with value: 0.8207834400816857.
[I 2026-02-22 19:13:48,682] Trial 1 finished with value: 0.8207741576162628 and parameters: {'C': 63.11926010142204, 'l1_ratio': 0.7141892210028146}. Best is trial 0 with value: 0.8207834400816857.
[I 2026-02-22 19:13:49,113] Trial 2 finished with value: 0.829555369906247 and parameters: {'C': 9.593762291833249, 'l1_ratio': 0.5556183740777708}. Best is trial 2 with value: 0.829555369906247.
[I 2026-02-22 19:13:49,695] Trial 3 finished with value: 0.8190104891859278 and parameters: {'C': 33.95378741976455, 'l1_ratio': 0.5902881068253787}. Best is trial 2 with value: 0.829555369906247.
[I 2026-02-22 19:13:50,148] Trial 4 finished with value: 0.8313097558711594 and parameters: {'C'

In [16]:
def RF_objective(trail):
    criterion = trail.suggest_categorical('criterion',['gini', 'entropy','log_loss'])
    min_samples_leaf = trail.suggest_int('min_samples_leaf', 2, 10)
    min_samples_split = trail.suggest_int('min_samples_split', 2, 15)
    max_samples = trail.suggest_float('max_samples', 0.6, 1)

    model = RandomForestClassifier(
        n_estimators=150,
        criterion = criterion,
        min_samples_leaf = min_samples_leaf,
        min_samples_split = min_samples_split,
        max_samples = max_samples,
        bootstrap=True,
        oob_score=True,
        n_jobs = -1,
        random_state=42
    )

    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score

In [17]:
study_RF = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_RF.optimize(RF_objective, n_trials=50)

[I 2026-02-22 19:14:39,484] A new study created in memory with name: no-name-c75ee41d-a3f6-4e36-b8dc-344129c5e6aa
[I 2026-02-22 19:14:41,457] Trial 0 finished with value: 0.8295739348370929 and parameters: {'criterion': 'entropy', 'min_samples_leaf': 8, 'min_samples_split': 6, 'max_samples': 0.9385442978506325}. Best is trial 0 with value: 0.8295739348370929.
[I 2026-02-22 19:14:43,117] Trial 1 finished with value: 0.8207834400816857 and parameters: {'criterion': 'gini', 'min_samples_leaf': 2, 'min_samples_split': 8, 'max_samples': 0.7481518332055882}. Best is trial 0 with value: 0.8295739348370929.
[I 2026-02-22 19:14:44,749] Trial 2 finished with value: 0.8260744453726909 and parameters: {'criterion': 'gini', 'min_samples_leaf': 3, 'min_samples_split': 2, 'max_samples': 0.8905026529965183}. Best is trial 0 with value: 0.8295739348370929.
[I 2026-02-22 19:14:46,497] Trial 3 finished with value: 0.8313376032674279 and parameters: {'criterion': 'log_loss', 'min_samples_leaf': 8, 'min_sa

In [18]:
def SVM_objective(trial):

    c = trial.suggest_float('C', 0.1, 20, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    degree = trial.suggest_int('degree', 1, 7)

    model = SVC(
            C=c, 
            kernel=kernel, 
            gamma=gamma, 
            degree=degree,
            random_state=42,
            probability=True
        )
    
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()

    return score

In [19]:
study_SVM = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_SVM.optimize(SVM_objective, n_trials=100)

[I 2026-02-22 19:16:05,179] A new study created in memory with name: no-name-6888e01b-128c-468e-94b3-ddf5297e7de0
[I 2026-02-22 19:16:05,415] Trial 0 finished with value: 0.6256637168141592 and parameters: {'C': 0.7905565624750227, 'kernel': 'poly', 'gamma': 'auto', 'degree': 6}. Best is trial 0 with value: 0.6256637168141592.
[I 2026-02-22 19:16:05,757] Trial 1 finished with value: 0.827775190187859 and parameters: {'C': 0.8942791077023287, 'kernel': 'rbf', 'gamma': 'auto', 'degree': 1}. Best is trial 1 with value: 0.827775190187859.
[I 2026-02-22 19:16:05,991] Trial 2 finished with value: 0.811939139885111 and parameters: {'C': 1.1664559181056744, 'kernel': 'poly', 'gamma': 'auto', 'degree': 1}. Best is trial 1 with value: 0.827775190187859.
[I 2026-02-22 19:16:06,416] Trial 3 finished with value: 0.8365626455519329 and parameters: {'C': 16.09875885404557, 'kernel': 'linear', 'gamma': 'scale', 'degree': 3}. Best is trial 3 with value: 0.8365626455519329.
[I 2026-02-22 19:16:06,642] T

In [20]:
best_svm_param = {'C': 1.3249868395495792, 'kernel': 'rbf', 'gamma': 'scale', 'degree': 5}

In [21]:
def GB_objective(trial):

    n_estimators = 200
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.05, log=True)
    max_depth = trial.suggest_int('max_depth', 18, 25)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()

    return score

In [22]:
study_GB = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_GB.optimize(GB_objective, n_trials=20)

[I 2026-02-22 19:16:30,972] A new study created in memory with name: no-name-a63403b9-e7cb-4bd1-b851-c31514ad867b
[I 2026-02-22 19:16:34,472] Trial 0 finished with value: 0.8207421207886973 and parameters: {'learning_rate': 0.021944312886615906, 'max_depth': 20, 'min_samples_split': 5, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.8207421207886973.
[I 2026-02-22 19:16:41,877] Trial 1 finished with value: 0.8101692283806863 and parameters: {'learning_rate': 0.026398693089385087, 'max_depth': 21, 'min_samples_split': 6, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.8207421207886973.
[I 2026-02-22 19:16:44,870] Trial 2 finished with value: 0.8435801894115821 and parameters: {'learning_rate': 0.01204749512114062, 'max_depth': 22, 'min_samples_split': 6, 'min_samples_leaf': 10}. Best is trial 2 with value: 0.8435801894115821.
[I 2026-02-22 19:16:49,170] Trial 3 finished with value: 0.8154789628939605 and parameters: {'learning_rate': 0.032738797168205304, 'max_depth': 24, 'm

In [23]:
# [I 2026-02-22 12:42:15,667] Trial 13 finished with value: 0.8426967398798386 and parameters: {'learning_rate': 0.01038067194767679, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 10}. Best is trial 13 with value: 0.8426967398798386.

In [24]:
def xgb_objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 7),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 1e-3, 5.0, log=True),
        
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'random_state': 42,
        'n_jobs': -1
    }

    model = XGBClassifier(**params)
    
    # StratifiedKFold is best here because the survival classes are slightly imbalanced
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy').mean()
    
    return score

In [25]:
study_XGB = optuna.create_study(direction='maximize')
study_XGB.optimize(xgb_objective, n_trials=100)

[I 2026-02-22 19:17:52,965] A new study created in memory with name: no-name-5a1fde58-e7dd-485d-aa76-e670bf01b5b1
[I 2026-02-22 19:17:53,870] Trial 0 finished with value: 0.8277441391088342 and parameters: {'max_depth': 5, 'min_child_weight': 4, 'gamma': 1.3014749712850846, 'subsample': 0.6080563094870303, 'colsample_bytree': 0.7432055697178924, 'n_estimators': 362, 'learning_rate': 0.01590576186274281, 'alpha': 0.02387961586641201, 'lambda': 0.005563195859493241}. Best is trial 0 with value: 0.8277441391088342.
[I 2026-02-22 19:17:54,567] Trial 1 finished with value: 0.827744139108834 and parameters: {'max_depth': 4, 'min_child_weight': 9, 'gamma': 0.5479806722781047, 'subsample': 0.8909330832810447, 'colsample_bytree': 0.728636923081998, 'n_estimators': 372, 'learning_rate': 0.08831144620731796, 'alpha': 0.07237199583113772, 'lambda': 0.13483175968718322}. Best is trial 0 with value: 0.8277441391088342.
[I 2026-02-22 19:17:55,258] Trial 2 finished with value: 0.8172178233193603 and p

In [26]:
study_RF.best_trial.params

{'criterion': 'entropy',
 'min_samples_leaf': 6,
 'min_samples_split': 3,
 'max_samples': 0.9851987586814172}

In [27]:
study_SVM.best_trial.params

{'C': 1.6963746629123329, 'kernel': 'rbf', 'gamma': 'scale', 'degree': 2}

In [28]:
study_XGB.best_trial.params

{'max_depth': 5,
 'min_child_weight': 1,
 'gamma': 0.01715518069488603,
 'subsample': 0.9439950277797844,
 'colsample_bytree': 0.8645354179897717,
 'n_estimators': 350,
 'learning_rate': 0.012894989466883576,
 'alpha': 0.8289901171806625,
 'lambda': 0.0028328551580498323}

In [29]:
study.best_trial.params

{'C': 3.3050079800374346, 'l1_ratio': 0.3775925570629637}

In [30]:
Rf = RandomForestClassifier(**study_RF.best_trial.params)
Gb = GradientBoostingClassifier(**study_GB.best_trial.params)
Svm = SVC(**study_SVM.best_trial.params)
Xgb = XGBClassifier(**study_XGB.best_trial.params)
Lr = LogisticRegression(**study.best_trial.params)

In [31]:
estimators = [("Lr",Lr),("Xgb",Xgb),("Rf",Rf),("Svm",Svm),("Gb",Gb)]

In [32]:
for estimator in estimators:
    x = cross_val_score(estimator[1],X_train,y_train,scoring = "accuracy", cv = 5)
    print(estimator[0], np.round(np.mean(x),2))

c:\Users\skmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\skmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\skmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\skmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\skmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1221:

Lr 0.85
Xgb 0.85
Rf 0.82
Svm 0.84
Gb 0.85


In [33]:
VotingEnsembleModel = VotingClassifier(estimators = estimators,voting = "hard")
x = cross_val_score(VotingEnsembleModel,X_train,y_train,cv=10,scoring='accuracy')
print(np.round(np.mean(x),2))

c:\Users\skmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\skmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\skmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\skmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\skmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1221:

0.85
